In [ ]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [8]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_notebook.yaml --gpus [0]

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
12:02:58   Random seed: 1024
12:02:58   Config file: config/recommender/pretrain_notebook.yaml
12:02:58   {'dataset': {'class': 'JointDataset',
             'graphs': ['Amazon_Beauty'],
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_item': {'dropout_rate': 0.1,
                                                 'hidden_dims': [32, 32],
                                                 'use_dropout': False,
                                                 'use_layer_norm': False},
                              'embedding_user': {'dropout_rate': 0.1,
                                                 'hidden_dims': [32, 32],
         

Load rspmm extension. This may take a while...


12:03:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:03:06   binary cross entropy: 0.704135
12:04:43   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:04:44   binary cross entropy: 0.605986


In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Amazon_Games --epochs 1 --bpe 10000 --gpus "[0]" --ckpt null

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 10 --bpe 50000 --gpus "[0]" --ckpt null

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
14:55:02   Random seed: 1024
14:55:02   Config file: config/recommender/notebook_cfg.yaml
14:55:02   {'checkpoint': None,
 'dataset': {'class': 'Yelp18',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_item': {'hidden_dims': [32, 32]},
                              'embedding_user': {'hidden_dims': [32, 32]},
                              'simple_model': {'aggregate_func': 'sum',
                                               'class': 'SimpleNBFNet',
                                               'hidden_dims': [40, 40, 40, 40],
                                               'input_dim': 40,
                                     

edge_attr.shape = torch.Size([2194014, 5])
x_user.shape = torch.Size([31668, 20])
x_item.shape = torch.Size([38048, 5850])


14:55:04   ------------------------------
14:55:04   Number of parameters: 54001
14:55:04   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:55:04   Epoch 0 begin


Load rspmm extension. This may take a while...
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])


14:55:05   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:55:05   Mean positive scores: -0.2981654703617096
14:55:05   Mean negative scores: -0.2990923225879669
14:55:05   BPR loss: 0.6926860213279724


...

relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size(

15:20:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])


15:20:52   Mean positive scores: 46.38694763183594
15:20:52   Mean negative scores: 40.72189712524414
15:20:52   BPR loss: 0.15426166355609894


relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size([69716, 320])
relation.shape() : torch.Size([2, 320])
edge_index.shape() : torch.Size([2, 2193998])
edge_type.shape() : torch.Size([2193998])
input.shape() : torch.Size(

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 4 --bpe 10 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_2025-01-08_15-24.pth

In [ ]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [7]:
import pickle


def inspect_cxtdict(cxtdict):
    print("Type:", type(cxtdict))
    print("Number of entries:", len(cxtdict))
    if isinstance(cxtdict, dict):
        print("Number of entries:", len(cxtdict))
        # Print a sample of the keys and values
        for i, key in enumerate(cxtdict.keys()):
            print(f"Key {i}: {key}, Value: {cxtdict[key]}")
            if i >= 9:  # Limit to first 10 entries
                break


def load_data(filename):
    try:
        with open(filename, "rb") as f:
            x= pickle.load(f)
    except:
        x = []
    return x
    
CXTDict = load_data('/itet-stor/trachsele/net_scratch/tl4rec/CARCA_Code_and_Data/Data/CXTDictSasRec_Games.dat')
inspect_cxtdict(CXTDict)
#print(CXTDict)
#print(CXTDict[(1,49583)])



Type: <class 'dict'>
Number of entries: 287107
Number of entries: 287107
Key 0: (352, 60), Value: [0.0, 0.5, 0.7419354838709677, 0.2857142857142857, 0.4767123287671233, 6.25]
Key 1: (1814, 647), Value: [0.0, 0.8333333333333334, 0.45161290322580644, 0.42857142857142855, 0.7863013698630137, 10.25]
Key 2: (1796, 619), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 3: (1504, 173), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 4: (1504, 364), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 5: (1504, 592), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 6: (1504, 785), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 7: (1504, 797), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.86027397

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
